# [MLPerf Inference v0.5](https://github.com/mlperf/inference/tree/master/v0.5) - results table generation

## Includes

In [ ]:
import os
import re
import json
from pprint import pprint

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
# import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
# print ('Seaborn version: %s' % sb.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

## Definitions

### Divisions, categories

In [ ]:
divisions = [ 'closed', 'open' ]
categories = [ 'available',  'preview', 'RDI', 'RDO' ]

### Maps for DataFrame construction

In [ ]:
# Lowercase or camelcase to camelcase.
scenario_to_str = {
    'SingleStream' : 'SingleStream',
    'singlestream' : 'SingleStream',

    'MultiStream'  : 'MultiStream',
    'multistream'  : 'MultiStream',

    'Server'       : 'Server',
    'server'       : 'Server',

    'Offline'      : 'Offline',
    'offline'      : 'Offline',
}

In [ ]:
# dividiti-specific.
system_id_to_processor = {
    'firefly'   : 'Rockchip RK3399',
    'hikey960'  : 'HiSilicon Kirin960',
    'mate10pro' : 'HiSilicon Kirin970',
    'rpi4'      : 'Broadcom BCM2711B0',
}

### Non-imagenet benchmarks

In [ ]:
non_imagenet_benchmarks = {
    # Non-ImageNet benchmarks from the closed division.
    'ssd-small': {
        "name"  : "SSD-MobileNet-v1",
        "width" : 300,
        "height": 300,
    },
    'ssd-large': {
        "name"  : "SSD-ResNet34",
        "width" : 1200,
        "height": 1200,
    },
    'gnmt' : {
        "name"  : "GNMT",
        "width" : -1,
        "height": -1,
    },
    # Non-ImageNet benchmarks from the open division.
    'rcnn-nas-lowproposals' : {
        "name" : "Faster-RCNN-NAS lowproposals",
        "url" : "http://download.tensorflow.org/models/object_detection/faster_rcnn_nas_lowproposals_coco_2018_01_28.tar.gz",
        "width" : 1200,
        "height" : 1200,
    },
    'rcnn-resnet50-lowproposals' : {
        "name" : "Faster-RCNN-ResNet50 lowproposals",
        "url" : "http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_lowproposals_coco_2018_01_28.tar.gz",
        "width" : 1024,
        "height" : 600,
    },
    'rcnn-resnet101-lowproposals' : {
        "name" : "Faster-RCNN-ResNet101 lowproposals",
        "url" : "http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_lowproposals_coco_2018_01_28.tar.gz",
        "width" : 1024,
        "height" : 600,
    },
    'rcnn-inception-resnet-v2-lowproposals' : {
        "name" : "Faster-RCNN-Inception-ResNet-v2 lowproposals",
        "url" : "http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco_2018_01_28.tar.gz",
        "width" : 1024,
        "height" : 600,
    },
    'rcnn-inception-v2' : {
        "name" : "Faster-RCNN Inception-v2",
        "url" : "http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz",
        "width" : 1024,
        "height" : 600,
    },
    'ssd-inception-v2' : {
        "name" : "SSD-Inception-v2",
        "url" : "http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz",
        "width" : 300,
        "height" : 300,
    },
    'ssd-mobilenet-v1-quantized-mlperf' : {
        "name" : "SSD-MobileNet-v1",
        "url" : "https://zenodo.org/record/3361502/files/ssd_mobilenet_v1_coco_2018_01_28.tar.gz",
        "width" : 300,
        "height" : 300,
        "provenance" : "Google",
    },
    'ssd-mobilenet-v1-non-quantized-mlperf' : {
        "name" : "SSD-MobileNet-v1 quantized",
        "url" : "https://zenodo.org/record/3252084/files/mobilenet_v1_ssd_8bit_finetuned.tar.gz",
        "width" : 300,
        "height" : 300,
        "provenance" : "Habana"
    },
    'ssd-mobilenet-v1-fpn' : {
        "name" : "SSD-MobileNet-v1 FPN SBP",
        "url" : "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03.tar.gz",
        "width" : 640,
        "height" : 640,
    },
    'ssd-resnet50-fpn' : {
        "name" : "SSD-ResNet50-v1 FPN SBP",
        "url" : "http://download.tensorflow.org/models/object_detection/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03.tar.gz",
        "width" : 640,
        "height" : 640,
    },
    'ssdlite-mobilenet-v2' : {
        "name" : "SSDLite-MobileNet-v2",
        "url" : "http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz",
        "width" : 300,
        "height" : 300,
    },
    'yolo-v3' : {
        "name" : "YOLO-v3",
        "url" : "https://zenodo.org/record/3386327/files/yolo_v3_coco.tar.gz",
        "width" : 416,
        "height" : 416,
        "provenance" : "https://github.com/YunYang1994/tensorflow-yolov3/"
    }
}

## Code

In [ ]:
results_path = '/home/anton/projects/mlperf/submissions_inference_0_5'

In [ ]:
dfs = []
for division in divisions:
    submitters_dir = os.path.join(results_path, division)
    if division != 'open': continue # skip closed for now.
    for submitter in os.listdir(submitters_dir):
        if submitter not in [ 'Qualcomm' ]: continue # [ 'dividiti', 'Qualcomm' ]
        systems_dir = os.path.join(submitters_dir, submitter, 'systems')
        results_dir = os.path.join(submitters_dir, submitter, 'results')
        pprint(results_dir)
        for system in os.listdir(results_dir):
            if system=='compliance_checker_log.txt': continue
            system_dir = os.path.join(results_dir, system)
            system_json_name = system+'.json'
            with open(os.path.join(systems_dir, system_json_name)) as system_json_file:
                system_json = json.load(system_json_file)
            for benchmark in os.listdir(system_dir):
                if benchmark=='compliance_checker_log.txt': continue
                benchmark_dir = os.path.join(system_dir, benchmark)
                # TODO: Iterate over scenarios.
                for scenario in os.listdir(benchmark_dir):
                    if scenario.lower() not in [ 'singlestream', 'multistream' ]: continue
                    experiment_dir = os.path.join(benchmark_dir, scenario)
                    # Extract accuracy in percent.
                    accuracy_dir = os.path.join(experiment_dir, 'accuracy')
                    with open(os.path.join(accuracy_dir, 'accuracy.txt'), 'r') as accuracy_file:
                        accuracy_txt = accuracy_file.readlines()
                        accuracy_line = accuracy_txt[-1]
                    if accuracy_line.startswith('mAP'):
                        task = 'OD'
                        match = re.match('mAP\=([\d\.]+)\%', accuracy_line)
                        accuracy_pc = float(match.group(1))
                    elif accuracy_line.startswith('accuracy'):
                        task = 'IC'
                        match = re.match('accuracy=(.+)%, good=(\d+), total=(\d+)', accuracy_line)
                        accuracy_pc = float(match.group(1))
                    else:
                        task = 'MT'
                    # Extract 90th percentile for SingleStream from the 6th line. TODO: Extract scores for the other scenarios.
                    performance_dir = os.path.join(experiment_dir, 'performance', 'run_1')
                    with open(os.path.join(performance_dir, 'mlperf_log_summary.txt'), 'r') as summary_file:
                        summary_txt = summary_file.readlines()
                        time_ns = int(summary_txt[6].split(':')[1].strip())
                        time_ms = time_ns * 1e-6
                    benchmark_dict = non_imagenet_benchmarks.get(benchmark)

                    # Extract input resolution.
                    if benchmark_dict:
                        width = benchmark_dict['width']
                        height = benchmark_dict['height']
                    else:
                        if benchmark.endswith('96'):
                            side = 96
                        elif benchmark.endswith('128'):
                            side = 128
                        elif benchmark.endswith('160'):
                            side = 160
                        elif benchmark.endswith('192'):
                            side = 192
                        else:
                            side = 224
                        width = side
                        height = side

                    # System details.
                    system_name = system_json['system_name']
                    system_list = system.split('-')
                    system_id = system_list[0]

                    # Processor (CPU).
                    processor = system_id_to_processor.get(system_id, system_json['host_processor_model_name'])
                    processor_num = system_json['host_processors_per_node']

                    # Accelerator.
                    accelerator = system_json['accelerator_model_name']
                    accelerator_num = system_json['accelerators_per_node']

                    # Software (framework).
                    software = system_json['framework']

                    # Benchmark (width x height).
                    benchmark_with_notes = '{} ({}x{})'.format(benchmark, width, height)

                    # Tasks.
                    ic1 = (task=='IC' and benchmark.startswith('mobilenet'))
                    ic2 = (task=='IC' and benchmark=='resnet')
                    od1 = (task=='OD' and benchmark=='ssd-mobilenet') # FIXME: benchmark==ssd-small?
                    od2 = (task=='OD' and system_id=='velociti')      # FIXME: benchmark==ssd-large?
                    nmt = False # TODO: Detect task and model.

                    # Default form factors and notes.
                    ff_m = ff_d = ff_s = ff_e = ''
                    notes = ''

                    # Submitter-specific form factors and notes.
                    submitter_str = submitter
                    if submitter == 'dividiti':
                        # Form factors.
                        if system_id in [ 'hikey960', 'firefly', 'rpi4' ]: ff_e = 'x'
                        if system_id in [ 'mate10pro', 'hikey960' ]: ff_m = 'x'
                        if system_id in [ 'velociti' ]: ff_d = 'x'                            
                        # Notes.
                        if system == 'velociti-tensorflow-v1.14-cpu':
                            notes = 'In the Other category, since this Intel CPU is no longer available (end-of-life).'
                        if system_id == 'hikey960':
                            notes = 'Mobile chip in embedded form factor (development board).'
                        # Object Detection is collaboration between dividiti and Politecnico di Milano.
                        if task == 'OD': submitter_str = 'dividiti + PoliMi'
                    elif submitter == 'Qualcomm':
                        notes = 'Median latency. MultiStream: Both Hexagon Vector Extensions (HVX) and Hexagon Tensor Accelerator (HTA).'
                            
                    # Prepare DataFrame data and index.
                    data = [{
                        # 
                        'ID': '-', # TODO: Fill in later.
                        'Submitter': submitter_str,
                        'System': system_name,
                        # TODO: Drop for Closed. Rename to 'Model used, if not Closed Division default' for Open.
                        'Benchmark': benchmark_with_notes,
                        # Performance metrics (FIXME: stream in ms, multistream in #streams, server in QPS, offline in inputs/s).
                        'P_IC1_SS' : '{:.03f}'.format(time_ms) if ic1 and scenario=='singlestream' else '',
                        'P_IC1_MS' : '{:.03f}'.format(time_ms) if ic1 and scenario=='multistream'  else '',
                        'P_IC1_S'  : '{:.03f}'.format(time_ms) if ic1 and scenario=='server'       else '',
                        'P_IC1_O'  : '{:.03f}'.format(time_ms) if ic1 and scenario=='offline'      else '',
                        'P_OD1_SS' : '{:.03f}'.format(time_ms) if od1 and scenario=='singlestream' else '',
                        'P_OD1_MS' : '{:.03f}'.format(time_ms) if od1 and scenario=='multistream'  else '',
                        'P_OD1_S'  : '{:.03f}'.format(time_ms) if od1 and scenario=='server'       else '',
                        'P_OD1_O'  : '{:.03f}'.format(time_ms) if od1 and scenario=='offline'      else '',
                        'P_IC2_SS' : '{:.03f}'.format(time_ms) if ic2 and scenario=='singlestream' else '',
                        'P_IC2_MS' : '{:.03f}'.format(time_ms) if ic2 and scenario=='multistream'  else '',
                        'P_IC2_S'  : '{:.03f}'.format(time_ms) if ic2 and scenario=='server'       else '',
                        'P_IC2_O'  : '{:.03f}'.format(time_ms) if ic2 and scenario=='offline'      else '',
                        'P_OD2_SS' : '{:.03f}'.format(time_ms) if od2 and scenario=='singlestream' else '',
                        'P_OD2_MS' : '{:.03f}'.format(time_ms) if od2 and scenario=='multistream'  else '',
                        'P_OD2_S'  : '{:.03f}'.format(time_ms) if od2 and scenario=='server'       else '',
                        'P_OD2_O'  : '{:.03f}'.format(time_ms) if od2 and scenario=='offline'      else '',
                        'P_NMT_SS' : ''                        if nmt and scenario=='singlestream' else '',
                        'P_NMT_MS' : ''                        if nmt and scenario=='multistream'  else '',
                        'P_NMT_S'  : ''                        if nmt and scenario=='server'       else '',
                        'P_NMT_O'  : ''                        if nmt and scenario=='offline'      else '',
                        # Accuracy metrics (Top1 for image classification, mAP for object detection, BLEU for machine translation).
                        # TODO: Drop for Closed.
                        'A_IC1_SS' : '{:.03f}'.format(accuracy_pc) if ic1 and scenario=='singlestream' else '',
                        'A_IC1_MS' : '{:.03f}'.format(accuracy_pc) if ic1 and scenario=='multistream'  else '',
                        'A_IC1_S'  : '{:.03f}'.format(accuracy_pc) if ic1 and scenario=='server'       else '',
                        'A_IC1_O'  : '{:.03f}'.format(accuracy_pc) if ic1 and scenario=='offline'      else '',
                        'A_OD1_SS' : '{:.03f}'.format(accuracy_pc) if od1 and scenario=='singlestream' else '',
                        'A_OD1_MS' : '{:.03f}'.format(accuracy_pc) if od1 and scenario=='multistream'  else '',
                        'A_OD1_S'  : '{:.03f}'.format(accuracy_pc) if od1 and scenario=='server'       else '',
                        'A_OD1_O'  : '{:.03f}'.format(accuracy_pc) if od1 and scenario=='offline'      else '',      
                        'A_IC2_SS' : '{:.03f}'.format(accuracy_pc) if ic2 and scenario=='singlestream' else '',
                        'A_IC2_MS' : '{:.03f}'.format(accuracy_pc) if ic2 and scenario=='multistream'  else '',
                        'A_IC2_S'  : '{:.03f}'.format(accuracy_pc) if ic2 and scenario=='server'       else '',
                        'A_IC2_O'  : '{:.03f}'.format(accuracy_pc) if ic2 and scenario=='offline'      else '',
                        'A_OD2_SS' : '{:.03f}'.format(accuracy_pc) if od2 and scenario=='singlestream' else '',
                        'A_OD2_MS' : '{:.03f}'.format(accuracy_pc) if od2 and scenario=='multistream'  else '',
                        'A_OD2_S'  : '{:.03f}'.format(accuracy_pc) if od2 and scenario=='server'       else '',
                        'A_OD2_O'  : '{:.03f}'.format(accuracy_pc) if od2 and scenario=='offline'      else '',      
                        'A_NMT_SS' : ''                            if nmt and scenario=='singlestream' else '',
                        'A_NMT_MS' : ''                            if nmt and scenario=='multistream'  else '',
                        'A_NMT_S'  : ''                            if nmt and scenario=='server'       else '',
                        'A_NMT_O'  : ''                            if nmt and scenario=='offline'      else '',
                        # Processor.
                        'Processor'     : processor,
                        'Processor #'   : processor_num,
                        # Accelerator.
                        'Accelerator'   : accelerator,
                        'Accelerator #' : accelerator_num if accelerator_num != '0' else '',
                        # Software.
                        'Software' : software,
                        # Form factor.
                        'FF_M'     : ff_m,
                        'FF_D'     : ff_d,
                        'FF_S'     : ff_s,
                        'FF_E'     : ff_e,
                        # Details. Code. Notes.
                        'Details'  : 'https://github.com/mlperf/submissions_inference_0_5/blob/master/{}/{}/systems/{}'. \
                                    format(division, submitter, system_json_name),
                        'Code'     : 'https://github.com/mlperf/submissions_inference_0_5/tree/master/{}/{}/code'. \
                                    format(division, submitter),
                        'Notes'    : notes,
                        # Misc.
                        'Division' : division,
                        'Category' : system_json['status'],
                        'Task'     : task,
                        'Scenario' : scenario_to_str.get(scenario, scenario),
                    }]
                    index = [
                        'Division', 'Category', 'Submitter', 'System', 'Task', 'Benchmark', 'Scenario', 'Software'
                    ]
                    # Construct a DataFrame.
                    df = pd.DataFrame(data)
                    df = df.set_index(index)
                    # Append to the list of similarly constructed DataFrames.
                    dfs.append(df)

                if dfs:
                    # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
                    df = pd.concat(dfs)
                    df.sort_index(ascending=False, inplace=True)
                    # Reset the index, but keep Division and Category there.
                    df = df.reset_index(level=index[2:])
                    # Mimic official template.
                    columns = [
                        'ID', 'Submitter', 'System', 'Benchmark',
                        'P_IC1_SS', 'P_IC1_MS', 'P_IC1_S', 'P_IC1_O',
                        'P_OD1_SS', 'P_OD1_MS', 'P_OD1_S', 'P_OD1_O',
                        'P_IC2_SS', 'P_IC2_MS', 'P_IC2_S', 'P_IC2_O',
                        'P_OD2_SS', 'P_OD2_MS', 'P_OD2_S', 'P_OD2_O',
                        'P_NMT_SS', 'P_NMT_MS', 'P_NMT_S', 'P_NMT_O',
                        'A_IC1_SS', 'A_IC1_MS', 'A_IC1_S', 'A_IC1_O',
                        'A_OD1_SS', 'A_OD1_MS', 'A_OD1_S', 'A_OD1_O',
                        'A_IC2_SS', 'A_IC2_MS', 'A_IC2_S', 'A_IC2_O',
                        'A_OD2_SS', 'A_OD2_MS', 'A_OD2_S', 'A_OD2_O',
                        'A_NMT_SS', 'A_NMT_MS', 'A_NMT_S', 'A_NMT_O',
                        'Processor'  , 'Processor #'  ,
                        'Accelerator', 'Accelerator #',
                        'Software',
                        'FF_M', 'FF_D', 'FF_S', 'FF_E',
                        'Details', 'Code', 'Notes'
                    ]
                    df = df[columns]

            # END OF FOR EACH benchmark
        # END OF FOR EACH system
    # END OF FOR EACH submitter
# END OF FOR EACH division

def link_code(url): return '<a target="_blank" href="{}">Code</a>'.format(url)
def link_details(url): return '<a target="_blank" href="{}">Details</a>'.format(url)
display_in_full(df.style.format({'Code': link_code, 'Details': link_details}))

# Create a Pandas Excel writer using XlsxWriter as the engine.
from pandas import ExcelWriter
xlsx_filename = 'submissions_inference_0_5.xlsx'
xlsx_writer = ExcelWriter(xlsx_filename, engine='xlsxwriter', options={'strings_to_urls': True})
for division in df.index.unique(level='Division'):
    df_d = df.loc[division]
    for category in df_d.index.unique(level='Category'):
        df_dc = df_d.loc[category]
        if division == 'open':
            df_xlsx = df_dc
        elif division == 'closed':
            # Show MobileNet and ResNet on the same row.
            df_mobilenet = df_dc[df_dc['Benchmark']=='mobilenet (224x224)']
            df_resnet    = df_dc[df_dc['Benchmark']=='resnet (224x224)']
            df_joined    = df_mobilenet.assign(
                P_IC2_SS=df_resnet['P_IC2_SS'].values,
                A_IC2_MS=df_resnet['A_IC2_SS'].values,
#                 P_IC2_MS=df_resnet['P_IC2_MS'].values,
#                 A_IC2_SS=df_resnet['A_IC2_MS'].values,
#                 P_IC2_S=df_resnet['P_IC2_S'].values,
#                 A_IC2_S=df_resnet['A_IC2_S'].values,                
#                 P_IC2_O=df_resnet['P_IC2_O'].values,
#                 A_IC2_O=df_resnet['A_IC2_O'].values,
            )
            accuracy_columns = [
                'A_{}_{}'.format(task, scenario)
                for task in ['IC1','OD1','IC2','OD2','NMT']
                for scenario in ['SS','MS','S','O']
            ]
            df_xlsx = df_joined.drop(labels=['Benchmark']+accuracy_columns, axis=1)
        else:
            continue
        # Write different division and category results to separate sheets. Omit index.
        df_xlsx.to_excel(xlsx_writer, sheet_name='{}-{}'.format(division, category), index=False)
        display_in_full(df_xlsx)
        
xlsx_writer.save()
!cp $xlsx_filename ~/Downloads